## Task Definition
- input:
- ouptut:

## RL

#### 環境設置

In [ ]:
import gym
import numpy as np
from gym import spaces

class AuditLogEnv(gym.Env):
    def __init__(self,logFile="G16.json"):
        # 初始化環境
        ## 基礎設定
        self.logFile=logFile
        #logguardQ有設置size但很明顯我要處理的檔案大很多因此不考慮
        self.stateDim=100 #代表傳入的狀態Dimension有100維
        self.actionDim=4 #代表有四種動作
        ## 接著是載入某份log(這邊預設維G16.json，因為它檔案很小)
        self.logs=self.loadAuditLogs(logFile)#這邊載入會使用 
        if len(self.logs) == 0:#沒有成功載入任何 AuditLog的話會出現錯誤
            raise ValueError(f"No valid log entries found in {logFile}.")
        self.attackPattern=[]#這邊logguardQ給了一個輸入維度在哪種情況下可能是攻擊，但我這邊還不確定怎麼做
        self.noiseLvl=0.05 # 一個加入可能的躁點大概佔幾趴 不確定會不會用到
        ## 以下是代理每次動作後看做了哪些事可能會更新某些值
        self.ipCounts={} # 由於AuditLog也有ip相關可能會用到
        self.logIndex=0 # 看到第幾條
        self.stepCount=0 # 現在走了幾步 
        self.anomalyCount=0
        self.visitedStates=set() # 走訪過那些狀態
        self.processedLogs()=set()# 已經處理了哪些東西 作用域: 單次回合 (Single Episode)
        self.processedLogsGlobal=set() #作用域: 整個環境的生命週期 (Entire Lifetime)。
        self.featureRanges={}# 把記錄到的某些數值縮放到某個數字區間 為了提高訓練效率和穩定性。 不確定會不會用到
        self.featureWeights={}# input的每個東西的權重，不確定要怎麼分配但先放著
        
        ## 常見弱點出現在哪些東西上
        vulnName=[]#常見的有問題的AuditLog可能的特徵 但這邊什麼都沒放 才剛架起來
        attackParamsValues=[]#logGuardQ使用常見的 XSS攻擊會出現的字串特徵 跟上面很像
        attackRegEx=[]# 仿造上面 這邊則用RegEx做做看 一樣還沒放入任何東西
        # 接下來是先載入一次這次回合的(Episode)的AuditLog 並且檢查有多少是非benign
        ## 它的主要功能是在環境載入完所有日誌後，對這些日誌進行初步的分析和統計，以瞭解數據集中異常日誌的「真實分佈」
        for
        self.reset()

    def reset(self):
        self.cursor = 0
        self.window_size = 5
        self.steps = 0

        self.labeled_logs = {}
        self.seen_entities = set()
        self.done = False

        return self._get_state()

    def step(self, action):
        self.steps += 1
        reward = -0.01  # 每一步都有成本

        if action == 0:
            pass  # SKIP

        elif 1 <= action <= 10:
            reward += self._handle_label(action)

        elif 11 <= action <= 15:
            reward += self._handle_search(action)

        elif 16 <= action <= 18:
            self._handle_expand(action)

        elif action == 19:
            reward += self._final_reward()
            self.done = True

        if self.steps >= self.max_steps:
            self.done = True

        return self._get_state(), reward, self.done, {}

    # ===== 以下是環境邏輯 =====

    def _handle_label(self, action):
        idx = (action - 1) // 2
        label = (action - 1) % 2  # 0 benign, 1 malicious

        candidates = self._topk_logs()
        if idx >= len(candidates):
            return -0.1

        log_id = candidates[idx]["id"]
        self.labeled_logs[log_id] = label

        # reward shaping（示意）
        return 0.2 if label == 1 else 0.05

    def _handle_search(self, action):
        idx = action - 11
        entities = self._topk_entities()
        if idx >= len(entities):
            return -0.1

        entity = entities[idx]
        self.seen_entities.add(entity)

        return 0.1

    def _handle_expand(self, action):
        if action == 16:
            self.cursor = max(0, self.cursor - 1)
        elif action == 17:
            self.cursor = min(len(self.logs) - self.window_size, self.cursor + 1)
        elif action == 18:
            self.cursor = max(0, self.cursor - 1)
            self.cursor = min(len(self.logs) - self.window_size, self.cursor + 1)

    def _final_reward(self):
        # session-level reward（F1 / consistency / LLM judge）
        return len(self.labeled_logs) * 0.1

    def _get_state(self):
        obs = self._extract_observation_features()
        mem = self._extract_memory_features()
        prog = np.array([
            self.steps / self.max_steps,
            len(self.labeled_logs),
            len(self.seen_entities)
        ])

        state = np.concatenate([obs, mem, prog])
        return state.astype(np.float32)

    # ===== Feature Extractors =====

    def _extract_observation_features(self):
        # 回傳固定長度向量（例如 40 維）
        return np.zeros(40)

    def _extract_memory_features(self):
        # 例如 21 維
        return np.zeros(21)

    def _topk_logs(self):
        return self.logs[self.cursor:self.cursor + self.window_size]

    def _topk_entities(self):
        # 從目前 window 抽 entity
        return []


#### 主要流程 